### Python module/package imports for this chapter

In [1]:
import sys, math, collections, itertools, multiprocessing, gzip, dis

In [2]:
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

%matplotlib inline

In [3]:
%load_ext line_profiler
%load_ext memory_profiler

In [17]:
import numba
from numba import jit
print(numba.__version__)
help(numba)
# from numba.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
# import warnings

# warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
# warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

0.51.2
Help on package numba:

NAME
    numba - Expose top-level symbols that are safe for import *

PACKAGE CONTENTS
    __main__
    _dispatcher
    _dynfunc
    _helperlib
    _version
    cext (package)
    core (package)
    cpython (package)
    cuda (package)
    experimental (package)
    extending
    misc (package)
    mviewbuf
    np (package)
    parfors (package)
    pycc (package)
    roc (package)
    runtests
    scripts (package)
    stencils (package)
    testing (package)
    tests (package)
    typed (package)
    types (package)

SUBMODULES
    config
    errors

CLASSES
    builtins.Exception(builtins.BaseException)
        numba.core.errors.NumbaError
            numba.core.errors.ByteCodeSupportError
            numba.core.errors.CompilerError
            numba.core.errors.ConstantInferenceError
            numba.core.errors.DeprecationError
            numba.core.errors.ForceLiteralArg
            numba.core.errors.IRError
                numba.core.errors.NotD

In [13]:
@jit
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    escaped = np.zeros((res,res),'d')  # escape status of points
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = compute_mandel(c,maxit)
    
    return escaped / maxit  # normalize for better plotting

@jit
def compute_mandel(c,maxit=256):
    z = 0.0j

    for it in range(1,maxit):
        z = z*z + c
        
        if abs(z) > 2.0:
            return it

    return np.inf  # no escape within maxit iterations... assume never

%lprun -f run_mandel run_mandel()


<ipython-input-13-8b09b93898f0>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "run_mandel" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(UniTuple(Literal[int](256) x 2), Literal[str](d))
 
There are 2 candidate implementations:
    - Of which 2 did not match due to:
    Overload of function 'zeros': File: numba/core/typing/npydecl.py: Line 504.
      With argument(s): '(UniTuple(int64 x 2), unicode_type)':
     No match.

During: resolving callee type: Function(<built-in function zeros>)
During: typing of call at <ipython-input-13-8b09b93898f0> (6)


File "<ipython-input-13-8b09b93898f0>", line 6:
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    <source elided>
    
    escaped = np.zeros((res,res),'d')  # escape status of points
    ^

  @jit
<ipython-input-13-8b09b93898f0>:1: NumbaWarning: 
Compilation is falling back to objec

In [14]:
@jit
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

@jit
def compute_mandel(c,maxit=256):
    z = 0.0j

    for it in range(1,maxit):
        z = z*z + c
        
        if abs(z) > 2.0:
            return it

    return np.inf  # no escape within maxit iterations... assume never

%lprun -f run_mandel run_mandel()

## Optimizing the computation of the Mandelbrot set 

In [ ]:
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    escaped = np.zeros((res,res),'d')  # escape status of points
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = compute_mandel(c,maxit)
    
    return escaped / maxit  # normalize for better plotting


def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

def compute_mandel(c,maxit=256):
    z = 0.0j

    for it in range(1,maxit):
        z = z*z + c
        
        if abs(z) > 2.0:
            return it

    return np.inf  # no escape within maxit iterations... assume never

In [ ]:
def plot_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256,run_mandel=run_mandel):
    pp.figure(figsize=(6,6))

    cmap = matplotlib.cm.coolwarm
    cmap.set_bad('k')

    pp.imshow(run_mandel(extent,res,maxit).T,extent=extent,
              cmap=cmap,norm=matplotlib.colors.PowerNorm(0.6),
              interpolation='none',origin='lower')

In [ ]:
%timeit run_mandel()

In [ ]:
0.0j

In [ ]:
np.seterr(all='ignore')
%timeit run_mandel_numpy()

In [ ]:
%prun run_mandel()

In [ ]:
%lprun -f run_mandel run_mandel()

In [ ]:
%lprun -f compute_mandel run_mandel()

In [ ]:
xs = np.linspace(extent[0],extent[1],res)
len(xs)

In [ ]:
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

In [ ]:
np.seterr(all='ignore')
%timeit run_mandel_numpy()

In [ ]:
%lprun -f run_mandel_numpy run_mandel_numpy()

In [ ]:
# print(run_mandel_numpy() * 256)

extent=(-2.0,1.0,-1.25,1.25)
res = 256
xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
len(c), len(c[0])
help(np.newaxis)
c

In [ ]:
plot_mandel(run_mandel=run_mandel_numpy)

In [ ]:
np.seterr(all='ignore')

In [ ]:
%timeit run_mandel_numpy()

In [ ]:
%lprun -f run_mandel_numpy run_mandel_numpy()

**March 2020 update**: recent version of `Numba` renamed `autojit` as `jit`. In addition, the Numba compiler has problems with functions that take lists and set (which are mutable) as input. In our case, it's easy to replace the `extent` list with a tuple. 

In [ ]:
@jit
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    escaped = np.zeros((res,res),dtype=np.float64)  # escape status of points
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = compute_mandel(c,maxit)
    
    return escaped / maxit  # normalize for better plotting

@jit
def compute_mandel(c,maxit=256):
    z = 0.0j

    for it in range(1,maxit):
        z = z*z + c
        
        if abs(z) > 2.0:
            return it

    return np.inf  # no escape within maxit iterations... assume never

In [ ]:
%timeit run_mandel()

In [ ]:
@jit
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

In [ ]:
np.seterr(all='ignore')
%timeit run_mandel_numpy()

In [ ]:
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

In [ ]:
%timeit run_mandel_numpy()

# 1. Noob python

In [ ]:
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    escaped = np.zeros((res,res),'d')  # escape status of points
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = compute_mandel(c,maxit)
    
    return escaped / maxit  # normalize for better plotting

%timeit run_mandel()

# 2 With numpy modifications

In [ ]:
from numba.decorators import jit

In [ ]:
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

%timeit run_mandel_numpy()

# 3 With modification and numba

In [ ]:
@jit('warn=False')
def run_mandel(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    escaped = np.zeros((res,res),'d')  # escape status of points
    
    for i in range(res):
        for j in range(res):
            c = xs[i] + 1j * ys[j]
            escaped[i,j] = compute_mandel(c,maxit)
    
    return escaped / maxit  # normalize for better plotting
# numba.seterr(all='ignore')
%timeit run_mandel()


# %timeit run_mandel_numpy()

In [ ]:
@jit
def run_mandel_numpy(extent=(-2.0,1.0,-1.25,1.25),res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)  # x and y arrays,
    ys = np.linspace(extent[2],extent[3],res)  # equally spaced
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]
    
    escaped = np.full_like(c,np.inf,'d')  # escape status of points

    
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = z*z + c
        
        escaped[(escaped == np.inf) & (np.abs(z) > 2.0)] = it
    
    return escaped / maxit  # normalize for better plotting

%timeit run_mandel_numpy()

In [ ]:
def twoSumApparentlySlow(nums, target):
    l = len(nums)
    if l < 2: return []
    for i in range(l-1):
        s1 = target-nums[i]
        if s1 in nums[i+1:]:
            j = nums[i+1:].index(s1)
            return [i, i+j+1]

In [ ]:
def twoSum(nums, target):
    l = len(nums)
    if l < 2: return []
    dct = {}
    for i in range(l):
        s1 = target-nums[i]
        if s1 in dct:
            j = dct[s1]
            return [j, i]
        dct[nums[i]] = i

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f twoSumApparentlySlow twoSumApparentlySlow([1,4,-2,-2,1], 0)

In [ ]:
%lprun -f twoSum twoSum([1,4,-2,-2,1], 0)

In [ ]:
%timeit twoSumApparentlySlow([1,4,-2,-2,1], 0)

In [ ]:
%timeit twoSum([1,4,-2,-2,1], 0)